In [5]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

# Modelling measures taken on COVID-19

Dr. Klaus G. Paul, Rolls-Royce Deutschland, R²Data Labs/Berlin AI Hub

## Summary

Assuming measures have to take place to drive the parameter $\mathit{7\,day\,growth\,infections}$ and $\mathit{14\,day\,growth\,infections}$, suggested here, below 2, it may become required to close operations to achieve this. Given this, it seems unlikely that relaxation of measures in D, UK, US takes place earlier than 36 days before this parameter hits and stays below 2.

## Description

This notebook attempts to model required further restrictions to public life and economy required to contain an uncontrolled spread of COVID-19. The author is not exactly an expert in virology.

We attempt to model the Basic reproduction number $R_0$, which appears to be the primary coefficient desribing spread of infectuous diseases. While it is required to keep $R_0 < 1$ to stop spread, the author assumes that governments will take action so as to immediately drive $R_0$ below 2 (to become comparable to cases of influenza, [wikipedia quotes $R_0$ between 2 and 3 for the 1918 pandemia](https://en.wikipedia.org/wiki/Basic_reproduction_number)). This notebook models $R_0$ as the percentage growth of new reported cases between seven (or 14) days, i.e. 

$$\mathit{7\,day\,growth\,infections} = \frac{active\,cases_{today}-active\,cases_{t-7 days}}{active\,cases_{t-7 days}}$$

$$\mathit{14\,day\,growth\,infections} = \frac{active\,cases_{today}-active\,cases_{t-14 days}}{active\,cases_{t-14 days}}$$

with

$$\mathit{active\,cases = n_{Confirmed} - n_{Deaths} - n_{Recovered}}$$

Seven or 14 days could be the median of days COVID-19 is infectuous without the patient showing symptoms. 

This assumption is not scientifically proven, the case in Italy (and Spain) show, however, that very drastic measures are taken if $\mathit{7\,day\,growth\,infections}$ is running away upward.

It is recommended to check absolute numbers and the evolution of data, not just the figures in this table. There is known limitations on the data quality, especially its completeness wrt the ability to provide completeness of and timely results on tests.

We conclude that measures to contain the spread will be extended until the $\mathit{n\,day\,growth\,infections}$ curves start to decline, and will stay in effect until $\mathit{n\,day\,growth\,infections}$ stabilizes (well) under 2. The number 2 is assumed to be in line with the capacity of the respective health care systems which regularly deal with seasonal influenza. From observations in China and Southg Korea it appears that there is impact on economics and daily life until these numbers stabilize at zero (0).

The purpose of this work is to detect a turning point at which measures have stabilized, i.e. limitations to public life and the ability to move (e.g., to work) are showing positive results allowing no firther measures to become necessary. 

Sources:
* Johns Hopkins CSSE data https://github.com/CSSEGISandData/COVID-19
* Basic reproduction number https://en.wikipedia.org/wiki/Basic_reproduction_number

## Summary

At the time of this writing, impact on the economy for RR cannot yet predicted. China crossed $\mathit{7\,day\,growth\,infections}$ threshold of 2 on 2020-02-08, the $\mathit{14\,day\,growth\,infections}$ threshold of 2 on 2020-02-17, and began to cautiously restart production around 2020-02-29 with putting workers under two week isolation after that. On 2020-03-15, there is reports of Foxconn beginning to set up production again. On 2020-03-24 there were reports that the Wuhan lockdown is cautiously lifted.

In [115]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook, output_file
from bokeh.models import Range1d, HoverTool, LinearAxis, Label
from bokeh.palettes import Category20, Category10
from bokeh.layouts import column
import random
from IPython.core.display import display, HTML
output_notebook()

Loading BokehJS ...

In [116]:
dfConfirmed = pd.read_csv("../csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
dfDeaths = pd.read_csv("../csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")
dfRecovered = pd.read_csv("../csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")

all_countries = set(dfConfirmed["Country/Region"])
subset_countries = ["Germany","United Kingdom","US","Singapore","India","Italy","Spain","China","Korea, South"]

acdsCountries = {}

for country in all_countries:
    dfCountry = dfConfirmed[dfConfirmed["Country/Region"] == country].transpose()
    columns = list(dfCountry.columns)
    dfCountry["date"] = pd.to_datetime(dfCountry.index,errors="coerce")
    dfCountry = dfCountry.dropna()
    dfCountry["confirmed"] = dfCountry[columns].sum(axis=1).astype(int)

    for c in columns:
        del dfCountry[c]
    
    ddf = dfDeaths[dfDeaths["Country/Region"] == country].transpose()
    columns = list(ddf.columns)
    ddf["date"] = pd.to_datetime(ddf.index,errors="coerce")
    ddf = ddf.dropna()
    ddf["deaths"] = ddf[columns].sum(axis=1).astype(int)

    for c in columns:
        del ddf[c]

    dfCountry = dfCountry.join(ddf,rsuffix = "_tmp")

    ddf = dfRecovered[dfRecovered["Country/Region"] == country].transpose()
    columns = list(ddf.columns)
    ddf["date"] = pd.to_datetime(ddf.index,errors="coerce")
    ddf = ddf.dropna()
    ddf["recovered"] = ddf[columns].sum(axis=1).astype(int)

    for c in columns:
        del ddf[c]

    dfCountry = dfCountry.join(ddf,rsuffix = "_tmp")
    del dfCountry["date_tmp"]

    dfCountry.fillna(0.,inplace=True)
    

    dfCountry["new_cases"] = dfCountry.confirmed.diff()
    dfCountry["growth_rate_3"] = dfCountry[["confirmed"]].pct_change(periods=3)
    dfCountry["growth_rate_7"] = dfCountry[["confirmed"]].pct_change(periods=7)
    dfCountry["growth_rate_14"] = dfCountry[["confirmed"]].pct_change(periods=14)

    dfCountry["active"] = dfCountry.confirmed-dfCountry.deaths-dfCountry.recovered
    dfCountry["new_infected"] = dfCountry.active.diff()
    dfCountry["infection_rate_3"] = dfCountry[["active"]].pct_change(periods=3)
    dfCountry["infection_rate_7"] = dfCountry[["active"]].pct_change(periods=7)
    dfCountry["infection_rate_14"] = dfCountry[["active"]].pct_change(periods=14)
    
    dfCountry["goal_2"] = 2

    relevant = dfCountry[dfCountry.confirmed > 0].index
    last_update = dfCountry.loc[relevant].date.max()
    acdsCountries[country] = ColumnDataSource(dfCountry.loc[relevant])

In [117]:
for country in subset_countries:
    pn = figure(plot_width=800, plot_height=200,x_axis_type='datetime')
    pn.vbar(x="date",top="new_cases",source=acdsCountries[country],
            width=86400*750,alpha=0.5,color=Category20[19][3],legend_label="New Cases")
    pn.title.text = "New Cases Reported in {} as of {:%Y-%m-%d %H:%M}".format(country,
                                                        pd.to_datetime(max(acdsCountries[country].data["date"])))
    pn.y_range.start=0
    pn.y_range.end=max(pd.Series(acdsCountries[country].data["new_cases"]).replace([np.inf, -np.inf], np.nan).dropna())*1.1

    pn.extra_y_ranges = {"active": Range1d(start=0, end=max(acdsCountries[country].data["active"]*1.1))}
    pn.line(x="date",y="active",source=acdsCountries[country],y_range_name="active",
            color=Category20[19][0],legend_label="Active Cases")
    pn.add_layout(LinearAxis(y_range_name="active"), 'left')

    pn.legend.location = "top_left"
    pn.legend.click_policy="hide"
    tooltips = [("date","@date{%Y-%m-%d}"),
                ("New Cases","@new_cases{0}"),
                ("Active Cases","@active{0}")]
    hover = HoverTool(tooltips = tooltips,
                      formatters={
                          "date": "datetime"
                      }
                     )
    pn.add_tools(hover)


    pg = figure(plot_width=800, plot_height=200,x_axis_type='datetime')
    pg.line(x="date",y="growth_rate_7",source=acdsCountries[country],
             color=Category20[19][4],legend_label="Growth Rate 7 day retrospective",name="growth_rate_7")
    pg.line(x="date",y="growth_rate_14",source=acdsCountries[country],
             color=Category20[19][0],legend_label="Growth Rate 14 day retrospective",name="growth_rate_14")
    pg.line(x="date",y="goal_2",source=acdsCountries[country],
             color=Category20[19][4],name="goal_2",line_dash="dashed")
    pg.title.text = "Infections Growth Rate Reported in {} as of {:%Y-%m-%d %H:%M}".format(country,
                                                        pd.to_datetime(max(acdsCountries[country].data["date"])))
    pg.legend.location = "top_left"
    pg.legend.click_policy="hide"
    pg.y_range.end = 10
    pg.y_range.start = 0
    tooltips = [("date","@date{%Y-%m-%d}"),
                ("7 days","@growth_rate_7{0.0}"),
                ("14 days","@growth_rate_14{0.0}")]
    hover = HoverTool(tooltips = tooltips,
                      formatters={
                          "date": "datetime"
                      }
                     )
    pg.add_tools(hover)


    pi = figure(plot_width=800, plot_height=200,x_axis_type='datetime')
    pi.line(x="date",y="infection_rate_7",source=acdsCountries[country],
             color=Category20[19][4],legend_label="Infection Rate 7 day retrospective",name="infection_rate_7")
    pi.line(x="date",y="infection_rate_14",source=acdsCountries[country],
             color=Category20[19][0],legend_label="Infection Rate 14 day retrospective",name="infection_rate_14")
    pi.line(x="date",y="goal_2",source=acdsCountries[country],
             color=Category20[19][4],name="goal_2",line_dash="dashed")
    pi.title.text = "New Infections Reported in {} as of {:%Y-%m-%d %H:%M}".format(country,
                                                        pd.to_datetime(max(acdsCountries[country].data["date"])))
    pi.legend.location = "top_left"
    pi.legend.click_policy="hide"
    pi.y_range.end = 10
    pi.y_range.start = 0
    tooltips = [("date","@date{%Y-%m-%d}"),
                ("7 days","@infection_rate_7{0.0}"),
                ("14 days","@infection_rate_14{0.0}")]
    hover = HoverTool(tooltips = tooltips,
                      formatters={
                          "date": "datetime"
                      }
                     )
    pi.add_tools(hover)

    display(HTML('<h2>{}</h2>'.format(country)))
    
    show(column([pn,pg,pi]))